# Imports and Scraping

In [92]:
import praw
import pytz
import datetime as dt

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pmaw import PushshiftAPI
api = PushshiftAPI()


def fxn(item):
    return (item['score'] > 0) & (item['over_18'] == False)
posts = api.search_submissions(filter_fn=fxn, subreddit="dominion", limit=50000)
post_list = [post for post in posts]

len(post_list)


Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift sh

7121

### Note that the length of posts is 7121 slightly less of the desired 10k.
Approval was granted by Caroline Schmitt on 9/6 to use lower amount of data.

## Examining titles/links of ten posts

In [105]:
df = pd.DataFrame(post_list)
start = 6480
inc = 10
for i in range(start,start+inc):
    print(f'i = {i}')
    print(df['title'].iloc[i])
    print(f'\t{df["full_link"].iloc[i]}')

i = 6480
U.S. Printer Issue?
	https://www.reddit.com/r/dominion/comments/38jhnu/us_printer_issue/
i = 6481
I need help deciding which expansion to get next
	https://www.reddit.com/r/dominion/comments/38eohs/i_need_help_deciding_which_expansion_to_get_next/
i = 6482
Interesting [Base/Prosperity/Seaside/Dark Ages/Adventures] Kingdom
	https://www.reddit.com/r/dominion/comments/38d8yr/interesting_baseprosperityseasidedark/
i = 6483
Are there any cards that you have NEVER bought?
	https://www.reddit.com/r/dominion/comments/3882v9/are_there_any_cards_that_you_have_never_bought/
i = 6484
Can you chain a remodel?
	https://www.reddit.com/r/dominion/comments/385pb3/can_you_chain_a_remodel/
i = 6485
KotW 5/31: Border Village, Cache, Highway, Horn of Plenty, Horse Traders, Mandarin, Menagerie, Remake, Scheme, Spice Merchant. [Cornucopia, Hinterlands]
	https://www.reddit.com/r/dominion/comments/381jer/kotw_531_border_village_cache_highway_horn_of/
i = 6486
I love Dominion, but I don't seem to under

## Examining data in one random post

In [50]:
list(zip(df.columns,df.iloc[3460]))

[('all_awardings', []),
 ('allow_live_comments', False),
 ('author', 'Knavenstine'),
 ('author_flair_css_class', None),
 ('author_flair_richtext', []),
 ('author_flair_text', None),
 ('author_flair_type', 'text'),
 ('author_fullname', 't2_2ldkc4qq'),
 ('author_is_blocked', nan),
 ('author_patreon_flair', False),
 ('author_premium', False),
 ('awarders', []),
 ('can_mod_post', False),
 ('contest_mode', False),
 ('created_utc', 1596144932),
 ('domain', 'self.dominion'),
 ('full_link',
  'https://www.reddit.com/r/dominion/comments/i0ur6i/sleeve_help/'),
 ('gildings', {}),
 ('id', 'i0ur6i'),
 ('is_created_from_ads_ui', nan),
 ('is_crosspostable', True),
 ('is_meta', False),
 ('is_original_content', False),
 ('is_reddit_media_domain', False),
 ('is_robot_indexable', True),
 ('is_self', True),
 ('is_video', False),
 ('link_flair_background_color', ''),
 ('link_flair_richtext', []),
 ('link_flair_text_color', 'dark'),
 ('link_flair_type', 'text'),
 ('locked', False),
 ('media_only', False),
 

### Examining data

In [128]:
df['num_comments'].value_counts()

0      1643
5       339
7       320
6       319
4       315
       ... 
88        1
71        1
110       1
59        1
104       1
Name: num_comments, Length: 103, dtype: int64

In [45]:
med = df['num_comments'].median()

In [46]:
df[df['num_comments'] <= med].shape

(3693, 101)

In [47]:
df[df['num_comments'] > med].shape

(3428, 101)

In [117]:
df['body_len'] = df['selftext'].apply(lambda val: 0 if type(val) == float else len(val))

In [118]:
df['has_media'] = df['media'].apply(lambda val: 1 if val == val else 0)

## Creating Time Columns

In [119]:
# Modified code from: https://stackoverflow.com/questions/52451105/python-converting-utc-millisecond-timestamp-to-local-time
def tz_from_utc_s_ts(utc_s_ts, tz_info):
    """Given second utc timestamp and a timezone return dateime

    :param utc_s_ts: Unix UTC timestamp in seconds
    :param tz_info: timezone info
    :return: timezone aware datetime
    """
    # convert from time stamp to datetime
    utc_datetime = dt.datetime.utcfromtimestamp(utc_s_ts)

    # set the timezone to UTC, and then convert to desired timezone
    return utc_datetime.replace(tzinfo=pytz.timezone('UTC')).astimezone(tz_info)

In [120]:
# Adding potentially useful columns
df['GMT_time'] = df['created_utc'].apply(lambda time_seconds: 
                                        tz_from_utc_s_ts(time_seconds, pytz.timezone('UTC')))
df['year'] = df['GMT_time'].apply(lambda date:  (date.strftime('%Y')))
df['month'] = df['GMT_time'].apply(lambda date:  (date.strftime('%m')))
df['day'] = df['GMT_time'].apply(lambda date:  (date.strftime('%d')))
df['hour'] = df['GMT_time'].apply(lambda date:  (date.strftime('%H')))

df['char_count'] = df['title'].apply(lambda text: len(text))
df['word_count'] = df['title'].apply(lambda text: len(text.split()))
med = df['num_comments'].median()
df['commments_greater'] = df['num_comments'].apply(lambda val: 1 if val > med else 0)

df.to_csv('../data/reddit_dominion_full.csv', index=False)
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,GMT_time,year,month,day,hour,char_count,word_count,commments_greater,body_len,has_media
0,[],False,Matcur12,None,[],None,text,t2_zpx31,False,False,...,2021-10-12 21:08:01+00:00,2021,10,12,21,63,10,0,808,0
1,[],False,fermalion,None,[],None,text,t2_m3ilj,False,False,...,2021-10-12 18:40:29+00:00,2021,10,12,18,55,11,1,1786,0
2,[],False,ABoyIsNo1,None,[],None,text,t2_1idri6nc,False,False,...,2021-10-12 17:18:24+00:00,2021,10,12,17,93,18,1,786,0
3,[],False,NovaRogue,None,[],Trash,text,t2_67a46,False,False,...,2021-10-12 15:53:08+00:00,2021,10,12,15,48,9,1,541,0
4,[],False,Augie279,None,[],None,text,t2_12gser,False,False,...,2021-10-11 17:29:05+00:00,2021,10,11,17,24,4,0,7389,0


In [121]:
df['commments_greater'].value_counts(normalize=True)

0    0.518607
1    0.481393
Name: commments_greater, dtype: float64

In [125]:
df.sort_values(by='created_utc', inplace=True)

In [126]:
df2 = df[['title',  'id', 'created_utc', 'year', 'month', 'full_link', 'body_len', 
          'word_count', 'char_count', 'has_media', 'num_comments', 
          'commments_greater']]
df2.head()

,title,id,created_utc,year,month,full_link,body_len,word_count,char_count,has_media,num_comments,commments_greater
7105,"Dominion Deck Builder | Share, discuss and rat...",en9hw,1292565223,2010,12,https://www.reddit.com/r/dominion/comments/en9...,0,11,74,0,0,0
7106,Dominion: Cornucopia announced. Theme: Variety.,f2gt5,1295040872,2011,01,https://www.reddit.com/r/dominion/comments/f2g...,0,5,47,0,1,0
7109,Online Dominion,fe1r5,1296680348,2011,02,https://www.reddit.com/r/dominion/comments/fe1...,0,2,15,0,3,0
7108,Looking for someone to play a multiple rounds,fj3la,1297381276,2011,02,https://www.reddit.com/r/dominion/comments/fj3...,274,8,45,0,6,0
7107,The Secret History of the Dominion Cards (writ...,fw55j,1299108955,2011,03,https://www.reddit.com/r/dominion/comments/fw5...,0,12,71,0,0,0


In [127]:
df2.to_csv('../data/reddit_dominion.csv', index=False)